In [ ]:
# Read files: format from Alex:
# speed:  [0, 2, 4, 6] 
# volume: [1, 3, 5, 7]
#  (samples, 4, 495, 436, 8), where in second dim: 
#         0: ground truth, 
#         1: point prediction, 
#         2: epistemic uncertainty, 
#         3: aleatoric uncertainty

import os
import h5py
from tqdm import tqdm
from smartprint import smartprint as sprint

model_list = ["unet", "unetpp"]
foldername_prefix = "h5_files_"
city_list = ["moscow", "antwerp"] 

for model in model_list:
    for city in city_list:
        filename = os.path.join(foldername_prefix + model, "pred_combo_" + city + ".h5") 

        with h5py.File(filename, "r") as f:
            a_group_key = list(f.keys())[0]
            data = list(f[a_group_key])
            sprint (len(data))
            
            for i in tqdm(range(len(data)), desc="Reading " + model + city):
                sample = data[i]
#                 sprint (sample.shape); break  # if just checking the shape


In [ ]:
from slugify import slugify

smoothing_window = 1000 # 1000 is a reasonable window considering the number of points \
                        # to be around 200K +. It can be increased even more; Plus we use valid padding
                        # whenever smoothing, so that no edge effects distort our view of the plots

chan = {"GT":0, "PP":1, "EU":2, "AU":3}

speed_channels = [0, 2, 4, 6]  # [4,5,6,7] # [0,1,2,3] # [1, 3, 5, 7]
volume_channels = [1, 3, 5, 7] # [0,1,2,3] # [4,5,6,7] # [0, 2, 4, 6]

density_label_flag = False
speed_thresh_low = 0.1
for model in model_list[:]:    
    for city in city_list[:]:
        filename = os.path.join(foldername_prefix + model, "pred_combo_" + city + ".h5") 

        X = []
        Y = [] 
        with h5py.File(filename, "r") as f:
            a_group_key = list(f.keys())[0]
            data = list(f[a_group_key])
            sprint (len(data))
            density_proxy = []
            sample_sum = 0 
            for i in tqdm(range(len(data))): #range(len(data)), desc="reading all files for one city"):
                sample = data[i]
                sample_sum += sample
        sample = sample_sum / len(data)
        
        sprint (sample.shape)
        
        volume_gt = sample[chan["GT"], :, :, volume_channels]
        volume_pred = sample[chan["PP"], :, :, volume_channels]
        volume_unc_a = sample[chan["AU"], :, :, volume_channels]
        volume_unc_e = sample[chan["EU"], :, :, volume_channels]
        
        speed_gt = sample[chan["GT"], :, :, speed_channels]
        speed_pred = sample[chan["PP"], :, :, speed_channels]
        speed_unc_a = sample[chan["AU"], :, :, speed_channels]
        speed_unc_e = sample[chan["EU"], :, :, speed_channels]        

        non_zero_speed = speed_gt > 0

        density_gt = volume_gt/speed_gt
        non_zero_density = density_gt > 0

        valid_indices = non_zero_density & non_zero_speed

        sprint (speed_gt.shape, non_zero_speed.shape, valid_indices.shape, density_gt.shape)
        sprint (volume_gt.shape, speed_gt.shape)
        speed_gt = speed_gt[valid_indices].tolist()
        speed_pred = speed_pred[valid_indices].tolist()
        speed_unc_a = speed_unc_a[valid_indices].tolist()
        speed_unc_e = speed_unc_e[valid_indices].tolist()

        volume_gt = volume_gt[valid_indices].tolist()
        volume_pred = volume_pred[valid_indices].tolist()
        volume_unc_a = volume_unc_a[valid_indices].tolist()
        volume_unc_e = volume_unc_e[valid_indices].tolist()

        density_gt = density_gt[valid_indices].tolist()
                
        

        ############ Same thing with absolute values of quantiles on the xticks
        def sort_two_lists_according_to_first(X,Y):
            ss = sorted(zip(X, Y))
            X = [x for x, y in ss]
            Y = [y for x, y in ss]    
            return X, Y


        import matplotlib.pyplot as plt
        import numpy as np




        X, _ = sort_two_lists_according_to_first(density_gt, speed_gt)
        plt.plot(X[smoothing_window//2:-smoothing_window//2], label="density")    

        names = ['speed_gt', 'speed_unc_a', 'speed_unc_e', 'volume_unc_a', 'volume_gt', 'volume_unc_e']
        for index, var in enumerate([speed_gt, speed_unc_a, speed_unc_e, volume_unc_a, volume_gt, volume_unc_e]):
            _, Y = sort_two_lists_according_to_first(density_gt, var)
            Y = Y[smoothing_window//2:-smoothing_window//2]

            plt.plot(np.convolve(Y, [1/smoothing_window]*smoothing_window, "same"), label=names[index])
        #     plt.show()
        plt.yscale("log")

        pos = np.array([0.0, 25.0, 50.0, 75.0, 95.0])
        perc = np.round(np.percentile(X, p), 2)

        plt.xticks(((len(X)-1) * pos/100), map(str, perc))
        
        plt.xlabel("Density proxy quantiles")
        plt.ylabel("Absolute values")
        plt.legend()
        plt.title(slugify(model+"_"+city))
        plt.show()
        sprint (len(var))




In [ ]:























def sort_two_lists_according_to_first(X,Y):
    ss = sorted(zip(X, Y))
    X = [x for x, y in ss]
    Y = [y for x, y in ss]    
    return X, Y


import matplotlib.pyplot as plt
import numpy as np


smoothing_window = 1101


X, _ = sort_two_lists_according_to_first(density_gt, speed_gt)
plt.plot(X[smoothing_window//2:-smoothing_window//2], label="density")    

names = ['speed_gt', 'speed_err', 'speed_unc', 'volume_err', 'volume_gt', 'volume_unc']
for index, var in enumerate([speed_gt, speed_unc_a, speed_unc_e, volume_unc_a, volume_gt, volume_unc_e]):
    
    sprint (len(var))
    _, Y = sort_two_lists_according_to_first(density_gt, var)
    Y = Y[smoothing_window//2:-smoothing_window//2]
    
    plt.plot(np.convolve(Y, [1/smoothing_window]*smoothing_window, "same"), label=names[index])
#     plt.show()
plt.yscale("log")
p = np.array([0.0, 25.0, 50.0, 75.0, 100.0])
plt.xticks(((len(X)-1) * p/100), map(str, p))
plt.legend()
plt.show()




############ Same thing with absolute values of quantiles on the xticks
def sort_two_lists_according_to_first(X,Y):
    ss = sorted(zip(X, Y))
    X = [x for x, y in ss]
    Y = [y for x, y in ss]    
    return X, Y


import matplotlib.pyplot as plt
import numpy as np


smoothing_window = 101


X, _ = sort_two_lists_according_to_first(density_gt, speed_gt)
plt.plot(X[smoothing_window//2:-smoothing_window//2], label="density")    

names = ['speed_gt', 'speed_unc_a', 'speed_unc_e', 'volume_unc_a', 'volume_gt', 'volume_unc_e']
for index, var in enumerate([speed_gt, speed_unc_a, speed_unc_e, volume_unc_a, volume_gt, volume_unc_e]):
    
    sprint (len(var))
    _, Y = sort_two_lists_according_to_first(density_gt, var)
    Y = Y[smoothing_window//2:-smoothing_window//2]
    
    plt.plot(np.convolve(Y, [1/smoothing_window]*smoothing_window, "same"), label=names[index])
#     plt.show()
plt.yscale("log")

pos = np.array([0.0, 25.0, 50.0, 75.0, 100.0])
perc = np.round(np.percentile(X, p), 2)

plt.xticks(((len(X)-1) * pos/100), map(str, perc))
plt.legend()
plt.show()



In [ ]:
################ 
################ NO need; 
################ the above cells suffice. the cells from here onwards;
################ are remnants of earlier trials;
################ 

# ! pip install smartprint
# ! pip install python-slugify
import numpy as np
import matplotlib.pyplot as plt 
from smartprint import smartprint
from slugify import slugify

def single_plot(a, y_list, density_channels, combine="mean", y_pred_name="GT", \
                speed_thresh_low=0):
    """
    a : numpy array of shape (4, 495, 436, 8)
    combine : ["mean", "mean_of_square"] # how to combine 4 channels
    y_list : list of y channels for plot; 
    density_channels : list of x channels for plot; Both x_list and y_list are from the the fourth dimension (index=3)
    speed_thresh_low : 0; threshold below which we ignore the density values (to avoid abnormally large values) 

    """
    chan = {"GT":0, "PP":1, "EU":2, "AU":3}
    speed_channels = [0, 2, 4, 6]

    
    density_vals = []
    non_zero_indices_in_density_channels = []
    X = []
    Y = []
    for index, i in enumerate(density_channels):
        
        # we need to filter zeros in speed channels to avoid np.nans in the density computation
        # The 0 in # a[0, ...] implies we are using the density computed using GT
        where_non_zero = a[0, :, :, speed_channels[index]] > speed_thresh_low
        
        # we use the indices where speed was non-zero to extract the densities, otherwise
        # we would get very high values for density
        x = a[0, :, :, i][where_non_zero]

        y = a[chan[y_pred_name], :, :, index][where_non_zero]

#         non_zero_indices_in_density_channels.append(where_non_zero)

#         if combine=="mean":
#             density_vals.extend( x.flatten().tolist() )
        X.extend(x.flatten().tolist())
        Y.extend(y.flatten().tolist())
    
#     density_vals

#     y = []
#     #  Now, we just need to choose the correponding Y values (GT, PP, EE, AU etc. for speed)
#     for index, i in enumerate(y_list):
#         x = a[chan[y_pred_name], :, :, i][non_zero_indices_in_density_channels[index]]
#         if combine=="mean":
#             y .extend( x.flatten().tolist() )

#     assert len(density_vals) == len(y)
    return X, Y
    

smoothing_window = 1 # 1000 is a reasonable window considering the number of points \
                        # to be around 200K + 
density_label_flag = False
speed_thresh_low = 0.1
for model in model_list[0:1]:    
    for city in city_list[0:1]:
        for traffic_var in ["Speed", "Volume"][:2]:
            for y_var in ["GT", "AU", "EU", "PP"][0:3]:                

                filename = os.path.join(foldername_prefix + model, "pred_combo_" + city + ".h5") 

                X = []
                Y = [] 
                with h5py.File(filename, "r") as f:
                    a_group_key = list(f.keys())[0]
                    data = list(f[a_group_key])
                    sprint (len(data))

                    density_proxy = []
                    for i in tqdm(range(20)): #range(len(data)), desc="reading all files for one city"):
                        sample = data[i]

                        ss = sample.shape

                        sample_with_density = np.random.rand(ss[0], ss[1], ss[2], 8 + 4) 
                        # + 4 is to store the 4 channels for the computed density

                        sample_with_density[:, :, :, :8] = np.array(sample)
                        
                        # mean_density is used to save the density computed by taking the 
                        # mean speed and volume across all channels
                        mean_density = np.random.rand(ss[1], ss[2]) 

                        for speed_channel in [0, 2, 4, 6]:
                            vol_channel = speed_channel + 1

                            # 0 in the first dimension is to because we don't care about the densities computed
                            # from anything apart from GT channels for speed and volume, all others are set to np.nan
                            # 8 + speed_channel//2 iterates over 8,9,10,11: The 4 channels where we store the 
                            # computed densities
                            sample_with_density[0, :, :, 8 + speed_channel//2] = \
                                        sample[0, :, :, vol_channel] / sample[0, :, :, speed_channel] 

                            for j in range(1, 4):
                                # all others are set to np.nan
                                sample_with_density[j, :, :, 8 + speed_channel//2] = np.nan
                        
                        gt_volume = (sample[0, :, :, 1] + sample[0, :, :, 3] + \
                                    sample[0, :, :, 5] + sample[0, :, :, 7])/4
                        gt_speed = (sample[0, :, :, 0] + sample[0, :, :, 2] + \
                                     sample[0, :, :, 4] + sample[0, :, :, 6])/4
                        
                        where_speed_more_than_speed_thresh = gt_speed > speed_thresh_low
                        mean_density[:,:] = gt_volume / gt_speed
                        mean_density = mean_density[where_speed_more_than_speed_thresh] 
                        gt_speed = gt_speed[where_speed_more_than_speed_thresh]
                                
                        
    #                     sprint (sample_with_density.shape)        

                        # y_list=[0,2,4,6] implies we wish to plot {AU, GT, EU, PP} w.r.t speed
                        # if we wish to plot w.r.t volume, we need to use y_list = [1,3,5,7]
                        y_list = {"Speed":[0,2,4,6], "Volume":[1,3,5,7]}
                        x,y = single_plot(sample_with_density, density_channels=[8,9,10,11], \
                                          y_list=y_list[traffic_var], \
                                          combine="mean", \
                                          y_pred_name=y_var,\
                                          speed_thresh_low=speed_thresh_low)

                        X.extend(x)
                        Y.extend(y)

#                         X.extend(mean_density.flatten().tolist())
#                         Y.extend(gt_speed.flatten().tolist())    

                X = np.array(X)
                Y = np.array(Y)
                
                X_no_nan = X[~np.isnan(X)]
                Y_no_nan = Y[~np.isnan(X)] 
                X = X_no_nan
                Y = Y_no_nan
                
                
                maxX = X.max()
                
                x = X[ (X<0.9 * maxX) & (X>5) ]
                y = Y[ (X<0.9 * maxX) & (X>5) ]
                X = x; Y = y
                
                X = X.flatten().tolist()
                Y = Y.flatten().tolist()
                
                ss = sorted(zip(X, Y))
                X = [x for x, y in ss]
                Y = [y for x, y in ss]        
                
                # X must be sorted
                assert all(X[i] <= X[i+1] for i in range(len(X) - 1))
                

                padded_length = max(smoothing_window, len(X)) - min(smoothing_window, len(X)) + 1
                # we just need the density plot once, we can just keep plotting it in the same color
                # but no need for the label so that we don't repeat
                if not density_label_flag:
                    plt.plot(range(padded_length), np.convolve(X, [1/smoothing_window]*smoothing_window, "valid"), \
                         label=slugify("density proxy sorted"), color="blue")
                    density_label_flag = True
                else:
                    plt.plot(range(padded_length), np.convolve(X, [1/smoothing_window]*smoothing_window, "valid"), \
                         color="blue")  # next time, no labels
                
                
                plt.plot(range(padded_length), np.convolve(Y, [1/smoothing_window]*smoothing_window, "valid"), \
                         label=y_var + "-" +traffic_var)
                plt.xlabel("Quantiles of density proxy")

                # add quantile x-ticks
                p = np.array([0.0, 25.0, 50.0, 75.0, 100.0])
                plt.xticks((len(X)-1) * p/100., map(str, p))

                plt.yscale("log")
                plt.legend()
                plt.title(slugify(city + "," + model))
        plt.show()




In [ ]:
plt.hist(X, 200, label="X")
plt.hist(Y, 200, label="Y")
plt.legend()
plt.show()

In [ ]:
plt.plot(range(len(X)), Y, alpha=0.1)
plt.yscale('log')
plt.show()

In [ ]:
Y.shape

model_list

In [ ]:
plt.scatter(X, Y, alpha=0.1, s=0.1)
plt.show()

In [ ]:
np.max(X), np.max(Y)

In [ ]:
all(Y[i] <= Y[i+1] for i in range(len(X) - 1))

In [ ]:
max(X), max(Y), len(X)

In [ ]:
plt.scatter(X, Y, alpha=0.01)
plt.show()

In [ ]:
x = X[(X>3) & (X<4)]
y = Y[(X>3) & (X<4)]
plt.plot(x, y)
plt.show()

In [ ]:
X.max()

In [ ]:
# ! pip install statsmodels
from statsmodels.graphics.gofplots import qqplot_2samples
qqplot_2samples(Y, X)

In [ ]:
X.shape, Y.shape